In [1]:
import pandas as pd
import os, glob
from owlready2 import *

import owlready2
print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

#```Merging ontologies:
#https://stackoverflow.com/questions/75102102/merge-two-ontologies-using-owlready
#```

0.40


In [2]:
onto = get_ontology("kg/pbn_t3_5.owl").load()

In [3]:
df = pd.read_parquet("outputs/newBP.parquet.gzip")
df.head(3)

,category,title,risks,subject,owner,mitig,techs,benefs,summary,details,...,summary_list,summ_context,summ_risk,summ_mitig,summ_benefits,det_recomm,det_who,det_scale,det_simple,SCALE
0,PBNCategory_50,"Implementing Regular Cleaning, WBE and Quarant...",This category of Risks describes various aspec...,"[Stakeholder_68, Stakeholder_9, Stakeholder_31...","[Stakeholder_269, Stakeholder_46, Stakeholder_...","[Mitigation_1587, Mitigation_988, Mitigation_1...","[Technology_22, Technology_358, Technology_185...","[Benef_5, Benef_1206, Benef_988]",* The Blueprint outlined addresses the pressin...,"* I recommend implementing the Blueprint, whic...",...,The Blueprint outlined addresses the pressing ...,The Blueprint outlined addresses the pressing ...,The risks associated with the prevailing COVID...,For the Blueprint to be effective in mitigatin...,The implementation of the Blueprint offers a p...,"I recommend implementing the Blueprint, which ...","Stakeholders such as healthcare institutions, ...",'Neighbourhood',The Blueprint lists numerous risks including t...,Neighbourhood
1,PBNCategory_97,Implementing Wastewater Monitoring and Enhance...,This category of Risks describes the potential...,"[Stakeholder_9, Stakeholder_2, Stakeholder_104...","[Stakeholder_1046, Stakeholder_413, Stakeholde...","[Mitigation_2162, Mitigation_2164, Mitigation_...","[Technology_3066, Technology_3067, Technology_...","[Benef_988, Benef_1206, Benef_5]",The context of this Blueprint revolves around ...,"* The ""Blueprint"" provides a comprehensive gui...",...,The context of this Blueprint revolves around ...,The context of this Blueprint revolves around ...,The risks identified within this blueprint foc...,The mitigation measures laid out in this bluep...,Implementing the mitigation measures laid out ...,"The ""Blueprint"" provides a comprehensive guide...","Wastewater treatment plant operators, research...",Neighbourhood,The blueprint addresses several risks related ...,Neighbourhood
2,PBNCategory_51,Implementing Effective Measures to Combat Cont...,This category of Risks describes a range of is...,"[Stakeholder_7, Stakeholder_562, Stakeholder_9...","[Stakeholder_627, Stakeholder_73, Stakeholder_...","[Mitigation_1853, Mitigation_2467, Mitigation_...","[Technology_22, Technology_23, Technology_2579...","[Benef_1366, Benef_632, Benef_631]",* The context of this blueprint derived from t...,* I strongly recommend implementing the 'Bluep...,...,The context of this blueprint derived from the...,The context of this blueprint derived from the...,The listed risks comprise domestic and intimat...,The blueprint underlines a strategic approach ...,The implementation of this blueprint holds sig...,I strongly recommend implementing the 'Bluepri...,"Government entities, NGOs, and law enforcement...",'Neighbourhood',The risks detailed in 'Blueprint' primarily co...,Neighbourhood


In [4]:
with onto:
    class aBlueprint(onto.Blueprint):
        pass
    class has_PBNCategoryTitle(onto.PBNCategory >> str):#owlready2.locstr):
        label = ["Title of the category"]
        pass
    class has_BPCategory(onto.Blueprint >> onto.PBNCategory ):#owlready2.locstr):
        label = ["Blueprints points to this category"]
        pass
    class has_CategoryBP(onto.PBNCategory >> onto.Blueprint ):#owlready2.locstr):
        label = ["Category contains the blueprint"]
        inverse_property = has_BPCategory
        pass
    class has_BlueprintSubject(onto.Blueprint >> onto.Stakeholder):#owlready2.locstr):
        label = ["Blueprint impacts these stakeholders"]
        pass
    class has_BlueprintOwner(onto.Blueprint >> onto.Stakeholder):#owlready2.locstr):
        label = ["Blueprint mitigation can be owned by these stakeholders"]
        pass
    class has_BlueprintMigitation(onto.Blueprint >> onto.Mitigation):#owlready2.locstr):
        label = ["Blueprint links to these mitigations"]
        pass
    class has_BlueprintTechnology(onto.Blueprint >> onto.Technology):#owlready2.locstr):
        label = ["Blueprint links to these technology"]
        pass
    class has_BlueprintBenef(onto.Blueprint >> onto.Benef):#owlready2.locstr):
        label = ["Blueprint links to these benefits"]
        pass

    class has_BlueprintSummary(onto.PBNCategory >> str):#owlready2.locstr):
        label = ["Summary of the blueprint"]
        pass
    class has_BlueprintDetails(onto.PBNCategory >> str):#owlready2.locstr):
        label = ["Details of the blueprint"]
        pass



In [5]:
for ix, row in df.iterrows():
    if ix < 5000:
        #print(ix)
        aBP = eval("onto.aBlueprint"+'("PBN__'+"aBlueprint"+"_"+str(ix)+'")')
        PBNCATEGORY = eval("onto.PBNCategory"+'("PBN__'+row["category"]+'")')
        aBP.has_BPCategory.append(PBNCATEGORY)
        aBP.label.append(row["title"])
        aBP.has_BlueprintSummary.append(row["summary"])
        aBP.has_BlueprintDetails.append(row["details"])
        
        for s in row["owner"]:
            S = "onto.Stakeholder"+'("PBN__'+s+'")'
            aBP.has_BlueprintOwner.append(eval(S))
        for s in row["subject"]:
            S = "onto.Stakeholder"+'("PBN__'+s+'")'
            aBP.has_BlueprintSubject.append(eval(S))
            
        for s in row["mitig"]:
            S = "onto.Mitigation"+'("PBN__'+s+'")'
            aBP.has_BlueprintMigitation.append(eval(S))
            
        for s in row["techs"]:
            S = "onto.Technology"+'("PBN__'+s+'")'
            aBP.has_BlueprintTechnology.append(eval(S))
            
        for s in row["benefs"]:
            S = "onto.Benef"+'("PBN__'+s+'")'
            aBP.has_BlueprintBenef.append(eval(S))

In [6]:
dfRisks = pd.read_parquet("outputs/CATS.parquet.gzip")
dfRisks
for ix, row in dfRisks.iterrows():
    if ix < 5000:
        PBNCATEGORY = eval("onto.PBNCategory"+'("PBN__'+row["category"]+'")')
        PBNCATEGORY.has_PBNCategoryTitle.append(row["CatDescr"])

In [7]:
onto.save("kg/new/pbn_t3_5.owl")